In [93]:
%load_ext autoreload

In [149]:
import polars as pl
from skolegpt_instruct_dataset.utils import sample_and_print_example
from skolegpt_instruct_dataset.translate import translate_with_deepl

df = pl.read_parquet("./data/stratified_dataset.parquet")

In [157]:
df_sub = df.sample(1)

In [161]:
from tqdm import tqdm
tqdm.pandas()

def translate_dataset(df: pl.DataFrame) -> pl.DataFrame:
    
    df = df.to_pandas()
    
    target_lang = "DA"  # For example, translating to German

    # Function to apply translation to a single column
    def translate_column(column):
        return column.map_elements(lambda x: translate_with_deepl(x, target_lang))

    # Apply the translation to the specified columns
    for col in ["system_prompt", "question", "response"]:
        df[col] = df[col].progress_apply(lambda x: translate_with_deepl(x, target_lang))

    df = pl.from_pandas(df)        

    return df

translate_dataset(df_sub)


id,system_prompt,question,response,source
str,str,str,str,str
"""t0.1232478""","""You are an AI …","""Please answer …","""In the movie ""…","""t0"""


In [151]:
df_sub

id,system_prompt,question,response,source
str,str,str,str,str
"""niv.318648""","""User will you …","""In this task y…","""To remove all …","""niv"""
"""flan.2153862""","""You are an AI …","""Answer the que…","""When the speak…","""flan"""
"""flan.1910888""","""You are an AI …","""Callimaco is t…","""No, the answer…","""flan"""
"""niv.279085""","""You are an AI …","""You will be gi…","""Incorrect answ…","""niv"""
"""cot.36088""","""You are an AI …","""Stream of cons…","""Question: Whic…","""cot"""
"""t0.1362540""","""You are an AI …","""I know that th…","""Insects want t…","""t0"""
"""t0.160725""","""You are an AI …","""Regular reader…","""100ml""","""t0"""
"""t0.202155""","""""","""Ricardo ""Ricky…","""Jarrett Jack""","""t0"""
"""niv.274908""","""Given a defini…","""You will be gi…","""Part #1: Given…","""niv"""


In [44]:

sample_and_print_example(df)

ID
niv.231718

SYSTEM PROMPT:

User will you give you a task with some instruction. Your job is
follow the instructions as faithfully as you can. While answering
think step-by-step and justify your answer.

QUESTION:

You will be given a definition of a task first, then some input of the
task. In this task, find the most appropriate number to replace the
blank (indicated with _ ) and express it in words.  Buttercups can
grow up to one to _ feet tall and can be about an inch across.

RESPONSE:

In order to find the most appropriate number to replace the blank,
let's look at the context of the sentence. We are given information
about Buttercups, including their height and size.  Since the height
is described in feet and the size in inches, it would be reasonable to
use a number within a typical range for the height of a plant. Let's
consider a number between 1 and 10 feet as this would be a reasonable
range for the height of Buttercups.  It's important to note that there
is no actual cor

id,system_prompt,question,response,source
str,str,str,str,str
"""niv.231718""","""User will you …","""You will be gi…","""In order to fi…","""niv"""


In [87]:
df_raw = pl.read_parquet("./data/sampled_dataset.parquet")

In [152]:
from 

ID
t0.595983

SYSTEM PROMPT:

You are an AI assistant. Provide a detailed answer so user don’t need
to search outside to understand the answer.

QUESTION:

Question: Information:  - In elementary geometry, a polytope is a
geometric object with flat sides, and may exist in any general number
of dimensions "n" as an "n"-dimensional polytope or "n"-polytope. For
example, a two-dimensional polygon is a 2-polytope and a three-
dimensional polyhedron is a 3-polytope.  - In computer science,
program optimization or software optimization is the process of
modifying a software system to make some aspect of it work more
efficiently or use fewer resources. In general, a computer program may
be optimized so that it executes more rapidly, or is capable of
operating with less memory storage or other resources, or draw less
power.  - In computer science and particularly in compiler design,
loop nest optimization (LNO) is an optimization technique that applies
a set of loop transformations for the pur

id,system_prompt,question,response,source
str,str,str,str,str
"""t0.595983""","""You are an AI …","""Question: Info…","""The polytope m…","""t0"""
